In [1]:
# dependencies
import pymongo as pm
import urllib
from scrape import Scrape as sc
from pprint import pprint
from bs4 import BeautifulSoup as bs
import re
import time

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pm.MongoClient(conn)

In [3]:
# define db and collections
db = client.OANN_scrape_db

# this collection will store all the HTML pages
htmlCollection = db.HTML_pages

In [4]:
# startURL = 'http://books.toscrape.com/'
# domain = 'toscrape.com'

# declare top-level url and domain
# startURL = 'https://billypenn.com/'
# domain = 'billypenn.com'

# startURL = 'https://theintercept.com/'
# domain = 'theintercept.com'

# startURL = 'https://www.wsj.com/'
# domain = 'wsj.com/'

# startURL = 'https://www.nationalreview.com/'
# domain = 'nationalreview.com/'

# startURL = 'https://www.economist.com/'
# domain = 'economist.com/'

# startURL = 'http://reason.com/'
# domain = 'reason.com'

startURL = 'http://www.oann.com/'
domain = 'oann.com'

# establish necessary data structures
done_urls = set()
external_urls = []
urls = set()
pagelinks = set()

# seed with top-level url of website
urls.add(startURL)

In [5]:
len(urls)

1

In [ ]:
# counter for limiting functions
ctr = 0

# continue to loop through urls until we have scraped all valid
# needed a while loop so we can modify size of urls through process
while (len(urls) > 0) and ctr < 10:    #(len(done_urls) < 200):
    
    # timer for performance across url loops
    forStart = time.time()
    
    # go through every url that has not been scraped yet
    for url in urls:
        print("URL in for loop: ", url)
        
        # timer for performance
        start = time.time()
        
        # governor to avoid being banned/blocked
        time.sleep(5)   # time governor
        
        # check for email and run all other links through scrape
        html = sc.scrapeHTML(url)
        
        # end timer for performance and print status
        end = time.time()
        print(f'Elapsed time: {end - start}')
        
        # validate html and then add to the pagelinks all the links that were scraped
        if html: 
            #print(f'Html scraped: {html[:50]}')
            data = {
                "url": url, 
                "html": html
                }

            # insert dict into db document
            try:
                htmlCollection.insert_one(data)
            except Exception as e:
                print(f'htmlCollection Insert: {e}')
                done_urls.add(url)
                continue
            
            # record url in "done" collection
            done_urls.add(url)
            print(f'done_urls: {len(done_urls)}')
            
            pagelinks = pagelinks.union(sc.scrapeURL(html, url))
            print(f'Extracted links: {len(pagelinks)}')
        else:
            done_urls.add(url)
            print(f'Main Loop: HTML not extracted')
        
        ####################################################
        # TODO: update done_urls, external and urls to CSV #
        ####################################################
        
    # end timer of for loop
    forEnd = time.time()
    
    # print status for testing
    print()
    print(f'Number of Urls: {len(urls)}')
    print(f'Process Time: {forEnd -forStart}')
    print('--------------------------------------------------------------')
        
        
    
    # add links from page to urls set outside of for loop
    if html:
        for link in pagelinks:
            if domain in link:
                urls.add(link)
                # print(f"Link in pagelinks: {link}")
            else:
                external_urls.append(link)
    
    # remove done_urls from todo urls and increment counter
    urls -= done_urls
    ctr += 1
    
    # print status for testing
    print("done_urls: ", len(done_urls))
    print("(urls - done_urls): ", len(urls))
    print("LOOP counter: ", ctr)
    print("--------------------")


URL in for loop:  http://www.oann.com/
Elapsed time: 5.329404592514038
done_urls: 1
Extracted links: 116

Number of Urls: 1
Process Time: 5.393886566162109
--------------------------------------------------------------
done_urls:  1
(urls - done_urls):  110
LOOP counter:  1
--------------------
URL in for loop:  http://www.oann.com/tag/g7-summit/
Elapsed time: 6.19235634803772
done_urls: 2
Extracted links: 123
URL in for loop:  http://www.oann.com/tag/israel/
Elapsed time: 6.292670011520386
done_urls: 3
Extracted links: 129
URL in for loop:  http://www.oann.com/tag/south-carolina/
Elapsed time: 5.179067611694336
done_urls: 4
Extracted links: 132
URL in for loop:  http://www.oann.com/schedule/
Elapsed time: 5.713348150253296
done_urls: 5
Extracted links: 132
URL in for loop:  http://www.oann.com/affiliaterelations/
Elapsed time: 6.050302505493164
done_urls: 6
Extracted links: 137
URL in for loop:  http://www.oann.com/supporters
Elapsed time: 6.489610910415649
done_urls: 7
Extracted link

Elapsed time: 6.115002393722534
done_urls: 62
Extracted links: 287
URL in for loop:  http://www.oann.com/?p=2313
Elapsed time: 6.523232460021973
done_urls: 63
Extracted links: 295
URL in for loop:  http://www.oann.com/thai-elephants-kick-off-anti-gambling-campaign-ahead-of-world-cup/
Elapsed time: 5.853017807006836
done_urls: 64
Extracted links: 295
URL in for loop:  http://www.oann.com/presskit/
Elapsed time: 5.720760345458984
done_urls: 65
Extracted links: 298
URL in for loop:  http://www.oann.com/tag/department-of-justice/
Elapsed time: 6.274375915527344
done_urls: 66
Extracted links: 299
URL in for loop:  http://www.oann.com/real-madrid-appoints-julen-lopetegui-as-coach-for-next-three-seasons/
Elapsed time: 6.712981700897217
done_urls: 67
Extracted links: 299
URL in for loop:  http://www.oann.com/tag/japan/
Elapsed time: 6.084884166717529
done_urls: 68
Extracted links: 301
URL in for loop:  http://www.oann.com/terms.html
Elapsed time: 5.143150329589844
done_urls: 69
Extracted links

Elapsed time: 5.836178541183472
done_urls: 120
Extracted links: 447
URL in for loop:  http://www.oann.com/former-heavyweight-champion-haye-announces-retirement/
Elapsed time: 6.035130977630615
done_urls: 121
Extracted links: 447
URL in for loop:  http://www.oann.com/russian-president-putin-confirms-he-wants-to-meet-with-pres-trump/
Elapsed time: 5.732839584350586
done_urls: 122
Extracted links: 450
URL in for loop:  http://www.oann.com/watch-house-lawmakers-alarmed-by-ig-report/
Elapsed time: 6.015215158462524
done_urls: 123
Extracted links: 450
URL in for loop:  http://www.oann.com/benghazi/
Elapsed time: 5.761749982833862
done_urls: 124
Extracted links: 450
URL in for loop:  http://www.oann.com/video/OAN30293_720p.mp4
Elapsed time: 213.33506751060486
htmlCollection Insert: BSON document too large (44299692 bytes) - the connected server supports BSON document sizes up to 16793598 bytes.
URL in for loop:  http://www.oann.com/former-white-house-adviser-steve-bannon-suggest-president-tru

Elapsed time: 16.557692050933838
done_urls: 173
Extracted links: 727
URL in for loop:  http://www.oann.com/tag/representative-bob-goodlatte/
Elapsed time: 5.994821786880493
done_urls: 174
Extracted links: 727
URL in for loop:  http://www.oann.com/google-to-work-with-pentagon-as-a-i-race-with-russia-china-accelerates/
Elapsed time: 6.017942190170288
done_urls: 175
Extracted links: 729
URL in for loop:  http://www.oann.com/category/economy/feed
Elapsed time: 6.351534843444824
done_urls: 176
Extracted links: 729
URL in for loop:  http://www.oann.com/doj-argues-parts-of-obamacare-are-unconstitutional-after-mandate-repeal/
Elapsed time: 6.031704425811768
done_urls: 177
Extracted links: 730
URL in for loop:  http://www.oann.com/australian-actress-rebel-wilson-has-bauer-media-libel-payout-cut/
Elapsed time: 5.787970542907715
done_urls: 178
Extracted links: 730
URL in for loop:  http://www.oann.com/president-trump-breaks-with-g7-joint-statement/
Elapsed time: 6.024256229400635
done_urls: 179
E

Elapsed time: 6.249703645706177
done_urls: 227
Extracted links: 896
URL in for loop:  http://www.oann.com/category/business/feed
Elapsed time: 6.3959267139434814
done_urls: 228
Extracted links: 896
URL in for loop:  http://www.oann.com/tag/video-game-disorder/
Elapsed time: 5.706175804138184
done_urls: 229
Extracted links: 896
URL in for loop:  http://www.oann.com/request
Elapsed time: 6.165517091751099
done_urls: 230
Extracted links: 905
URL in for loop:  http://www.oann.com/u-s-treasury-to-slap-secondary-sanctions-on-eu-over-iran-ties/
Elapsed time: 5.7920026779174805
done_urls: 231
Extracted links: 907
URL in for loop:  http://www.oann.com/one-america-news-appoints-emerald-robinson-as-chief-white-house-correspondent/
Elapsed time: 5.851170778274536
done_urls: 232
Extracted links: 913
URL in for loop:  http://www.oann.com/category/money/page/5/
Elapsed time: 5.797861576080322
done_urls: 233
Extracted links: 924
URL in for loop:  http://www.oann.com/category/shows/escape-from-iran/
El

...

Elapsed time: 5.729481935501099
done_urls: 1733
Extracted links: 1957
URL in for loop:  http://www.oann.com/watch-roseanne-barr-stripped-of-career-and-reputation-for-social-media-posts/
Elapsed time: 6.064619064331055
done_urls: 1734
Extracted links: 1958
URL in for loop:  http://www.oann.com/category/business/page/9/
Elapsed time: 6.351433277130127
done_urls: 1735
Extracted links: 1966
URL in for loop:  http://www.oann.com/samsung-confirms-chinese-officials-visited-sales-offices/
Elapsed time: 6.124279022216797
done_urls: 1736
Extracted links: 1966
URL in for loop:  http://www.oann.com/tag/berto-capo/
Elapsed time: 6.054676532745361
done_urls: 1737
Extracted links: 1966
URL in for loop:  http://www.oann.com/u-n-rights-boss-takes-aim-at-u-s-migration-policy-china-myanmar-in-final-speech/
Elapsed time: 6.156943082809448
done_urls: 1738
Extracted links: 1966
URL in for loop:  http://www.oann.com/tag/venezuela/
Elapsed time: 5.7722156047821045
done_urls: 1739
Extracted links: 1966
URL in 

Elapsed time: 5.959841251373291
done_urls: 1783
Extracted links: 1987
URL in for loop:  http://www.oann.com/australian-regulator-sues-westpac-bank-claiming-poor-financial-advice/
Elapsed time: 5.862728595733643
done_urls: 1784
Extracted links: 1987
URL in for loop:  http://www.oann.com/euro-falls-one-percent-versus-dollar-and-bond-yields-tumble-on-dovish-ecb-interest-rate-guidance/
Elapsed time: 5.858051061630249
done_urls: 1785
Extracted links: 1987
URL in for loop:  http://www.oann.com/u-s-officials-warn-congress-on-risks-of-drones-seek-new-powers/
Elapsed time: 5.807045936584473
done_urls: 1786
Extracted links: 1987
URL in for loop:  http://www.oann.com/turkeys-jailed-pro-kurdish-candidate-in-first-tv-appearance-for-20-months/
Elapsed time: 6.092974901199341
done_urls: 1787
Extracted links: 1987
URL in for loop:  http://www.oann.com/u-s-returns-stolen-525-year-old-columbus-letter-to-vatican/
Elapsed time: 6.088468313217163
done_urls: 1788
Extracted links: 1987
URL in for loop:  http

Elapsed time: 5.78335428237915
done_urls: 1832
Extracted links: 1997
URL in for loop:  http://www.oann.com/australian-pm-steps-into-world-cup-streaming-row/
Elapsed time: 6.19028902053833
done_urls: 1833
Extracted links: 1997
URL in for loop:  http://www.oann.com/french-central-bank-trims-2018-growth-forecast-to-1-8-percent/
Elapsed time: 6.14087986946106
done_urls: 1834
Extracted links: 1997
URL in for loop:  http://www.oann.com/deutsche-bank-sells-1-billion-non-performing-ship-loan-portfolio-sources/
Elapsed time: 6.147669792175293
done_urls: 1835
Extracted links: 1997
URL in for loop:  http://www.oann.com/china-promises-fast-response-if-u-s-hurts-its-interests/
Elapsed time: 6.273972511291504
done_urls: 1836
Extracted links: 1997
URL in for loop:  http://www.oann.com/boj-keeps-policy-steady-cuts-view-on-consumer-inflation/
Elapsed time: 6.811342000961304
done_urls: 1837
Extracted links: 1997
URL in for loop:  http://www.oann.com/federal-judge-orders-wash-prison-to-give-inmates-ramad

Elapsed time: 5.7891974449157715
done_urls: 1883
Extracted links: 2026
URL in for loop:  http://www.oann.com/a-northern-chinese-powerhouse-grapples-with-a-slowdown-as-credit-curbs-hit/
Elapsed time: 6.129499912261963
done_urls: 1884
Extracted links: 2026
URL in for loop:  http://www.oann.com/russian-police-detain-british-lgbt-rights-campaigner-in-moscow/
Elapsed time: 6.120429277420044
done_urls: 1885
Extracted links: 2026
URL in for loop:  http://www.oann.com/european-shares-open-lower-before-ecb-meeting/
Elapsed time: 6.3182244300842285
done_urls: 1886
Extracted links: 2026
URL in for loop:  http://www.oann.com/u-s-says-internet-use-rises-as-more-low-income-people-go-online/
Elapsed time: 6.154689788818359
done_urls: 1887
Extracted links: 2026
URL in for loop:  http://www.oann.com/new-zealand-announces-tourist-tax-to-fund-straining-infrastructure/
Elapsed time: 6.085183143615723
done_urls: 1888
Extracted links: 2026
URL in for loop:  http://www.oann.com/indonesians-eid-balloons-force

Elapsed time: 6.049978256225586
done_urls: 1931
Extracted links: 2055
URL in for loop:  http://www.oann.com/u-s-vice-president-pence-to-visit-guatemala-volcano-victims-white-house/
Elapsed time: 6.247515439987183
done_urls: 1932
Extracted links: 2055
URL in for loop:  http://www.oann.com/tag/representative-keith-ellison/
Elapsed time: 5.8295369148254395
done_urls: 1933
Extracted links: 2055
URL in for loop:  http://www.oann.com/three-people-hurt-in-moscow-taxi-incident-to-be-discharged-from-hospital-ria/
Elapsed time: 5.938725471496582
done_urls: 1934
Extracted links: 2055
URL in for loop:  http://www.oann.com/tag/cbp/
Elapsed time: 6.791375160217285
done_urls: 1935
Extracted links: 2055
URL in for loop:  http://www.oann.com/category/world/page/12/
Elapsed time: 5.937257766723633
done_urls: 1936
Extracted links: 2064
URL in for loop:  http://www.oann.com/ztes-hong-kong-listed-shares-slide-as-much-as-10-percent/
Elapsed time: 5.814092636108398
done_urls: 1937
Extracted links: 2064
URL i

In [ ]:
counter = 0
for url in urls:
    print(f'URL #{counter}: {url}')